<a href="https://colab.research.google.com/github/AlexanderCardarasUCSC/EEG-Decoding-to-Images/blob/main/CSE_247_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install packages for running on PRP**

In [1]:
!pip -q install keras
!pip -q install google-cloud-storage
!pip -q install google-api-python-client oauth2client

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None


In [2]:
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools

obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 
        'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the
# user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

/opt/conda/lib/python3.8/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access token.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=690369448729-2hvau5npdbarjcoenoqb4nr2g7mvih5e.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

Enter verification code: 4/1AY0e-g4-WoAqxi9kGuJTGuvI1RTERIdNhNtWpIdbW3J9hvTWmdl2jU2H3B4
Authentication successful.


In [3]:
import io
from googleapiclient.http import MediaIoBaseDownload

DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))

def download_file(file_id, filename):
    # if you get the shareable link, the link contains this id, replace the file_id below
    request = DRIVE.files().get_media(fileId=file_id)
    # replace the filename and extension in the first field below
    fh = io.FileIO(filename, mode='w')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

In [4]:
# Download Image data
!mkdir -p data/images
!wget -q https://raw.githubusercontent.com/AlexanderCardarasUCSC/EEG-Decoding-to-Images/main/image_utils.py 
download_file("1-2WFgtBn5WdZCnDlJkwaswTTrRtVNrS0", "images.npy")
download_file("1Lijm9A5Kq4EMub5jO0Bzz3c2VJLm-WH6", "image_labels.npy")

!sudo mv images.npy data/images
!sudo mv image_labels.npy data/images

Download 23%.
Download 47%.
Download 71%.
Download 94%.
Download 100%.
Download 100%.


# EEG decoding to images - Load data

**Complete the following steps before running the cells below**

1.
Change runtime to GPU 

>Runtime > Change runtime type > GPU

2.
For each of the following link, add a Google Drive shortcut to your Drive root folder(My Drive):

>https://drive.google.com/drive/folders/1z8wcew5R7GCeu5s7SSsWw8bUcx1urVmk?usp=sharing

**Mount drive**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

**Run the following cells to load eeg and/or image data**

#### Load eeg data

**Uncomment one of the following blocks**

*5-95Hz*

In [ ]:
# !mkdir -p /content/data/eeg
# !cp /content/gdrive/MyDrive/EEG2Image/eeg_5_95_std.pth /content/data/eeg/
# eeg_path = "/content/data/eeg/eeg_5_95_std.pth"

14-70Hz

In [ ]:
# !mkdir -p /content/data/eeg
# !cp /content/gdrive/MyDrive/EEG2Image/eeg_14_70_std.pth /content/data/eeg/
# eeg_path = "/content/data/eeg/eeg_14_70_std.pth"

*55-95Hz*

In [ ]:
!mkdir -p /content/data/eeg
!cp /content/gdrive/MyDrive/EEG2Image/eeg_54_95_std.pth /content/data/eeg/
eeg_path = "/content/data/eeg/eeg_54_95_std.pth"

#### Load image data

In [ ]:
!mkdir -p /content/data/images
!wget -q https://raw.githubusercontent.com/AlexanderCardarasUCSC/EEG-Decoding-to-Images/main/image_utils.py /content
!cp /content/gdrive/MyDrive/EEG2Image/data/images.npy /content/data/images
!cp /content/gdrive/MyDrive/EEG2Image/data/image_labels.npy /content/data/images

**Example load image**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import image_utils

In [ ]:
def load_images(path_root="/content/"):
  images = np.load(path_root+"data/images/images.npy")
  image_labels = np.load(path_root+"data/images/image_labels.npy")
  return images, image_labels

In [ ]:
images, labels = image_utils.load_images(path_root="")

In [ ]:
print(images.shape)
plt.imshow(images[0])

# Baseline GAN

**Complete the following steps before running the cells below**

1. Follow directions in the Load data section to load image data

>Run the cells under the header **Load image data**

**Import libraries**

### Jason's AC-GAN 

In [ ]:
# example of fitting an auxiliary classifier gan (ac-gan) on fashion mnsit
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
import image_utils

**Discriminator architecture**

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(64,64,3), n_classes=10):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=in_shape)
	# downsample to 14x14
	fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# normal
	fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# downsample to 7x7
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU( alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# normal
	fe = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(fe)
	fe = BatchNormalization()(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.5)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# real/fake output
	out1 = Dense(1, activation='sigmoid')(fe)
	# class label output
	out2 = Dense(n_classes, activation='softmax')(fe)
	# define model
	model = Model(in_image, [out1, out2])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model


**Generator architecture**

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=10):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	# li = Embedding(n_classes, 50)(in_label)
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	# n_nodes = 7 * 7
	n_nodes = 16 * 16 * 3
	li = Dense(n_nodes, kernel_initializer=init)(li)
	# reshape to additional channel
	# li = Reshape((7, 7, 1))(li)
	li = Reshape((16, 16, 3))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	# n_nodes = 384 * 7 * 7
	n_nodes = 384 * 16 * 16
	gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
	gen = Activation('relu')(gen)
	# gen = Reshape((7, 7, 384))(gen)
	gen = Reshape((16, 16, 384))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
	gen = BatchNormalization()(gen)
	gen = Activation('relu')(gen)
	# upsample to 28x28
	# gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
	gen = Conv2DTranspose(3, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
	out_layer = Activation('tanh')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

**GAN architecture**

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect the outputs of the generator to the inputs of the discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and label and outputting real/fake and label outputs
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.2)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model


**Helper functions**

In [ ]:
def trim_classes(images, labels, max_classes=10):
  return images[0:900*max_classes], labels[0:900*max_classes]

# splits image data into training and test sets(currently no randomization)
def new_split_data(images, labels, train=0.8, data_per_class=900):
  trainX,trainY,testX,testY = [],[],[],[]
  for i in range(images.shape[0]):
    chunk = i % data_per_class
    if chunk < data_per_class*train:
      trainX.append(images[i])
      trainY.append(labels[i])
    else:
      testX.append(images[i])
      testY.append(labels[i])

  return (np.asarray(trainX), np.asarray(trainY)), (np.asarray(testX), np.asarray(testY))

# load images
def load_real_samples(n_classes):
  # load dataset
  images, labels = image_utils.load_images(path_root="")
  images, labels = trim_classes(images, labels, n_classes)
  (trainX, trainY), (testX, testY) = new_split_data(images, labels)
	# expand to 3d, e.g. add channels
  X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
  X = X.astype('float32')
	# scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  print(X.shape, trainY.shape)
  return [X, trainY]
 
# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes):
  # generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  # reshape into a batch of inputs for the network
  z_input = x_input.reshape(n_samples, latent_dim)
  # generate labels
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples, n_classes)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

def generate_fake_class_sample(generator, class_label, latent_dim, n_samples):
  # generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  # reshape into a batch of inputs for the network
  z_input = x_input.reshape(n_samples, latent_dim)
  labels_input = np.asarray([class_label for i in range(n_samples)])
  # predict outputs
  images = generator.predict([z_input, labels_input])
  # create class labels
  y = zeros((n_samples, 1))
  return [images, labels_input], y
 
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	[X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		# pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
		pyplot.imshow(X[i, :, :])
	# save plot to file
	filename1 = 'generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_classes=10):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		_,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
		# generate 'fake' examples
		[X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		_,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
		# prepare points in latent space as input for the generator
		[z_input, z_labels] = generate_latent_points(latent_dim, n_batch, n_classes)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		_,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
		# summarize loss on this batch
		print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
		# evaluate the model performance every 'epoch'
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, latent_dim)

**Train GAN**

In [ ]:
# size of the latent space
# latent_dim = 100
latent_dim = 100
n_classes=1
# create the discriminator
discriminator = define_discriminator(n_classes=n_classes)
# create the generator
generator = define_generator(latent_dim, n_classes)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples(n_classes)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_classes=n_classes)

In [ ]:
[X, _], _ = generate_fake_class_sample(generator, class_label=0, latent_dim=latent_dim, n_samples=16)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0

f, axs = plt.subplots(4,4,figsize=(15,15))
# plot images
for i in range(16):
  axs[i//4,i%4].imshow(X[i])
  axs[i//4,i%4].axis("off")
f.show()


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(generator, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)
plot_model(discriminator, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

In [37]:
# example of training the discriminator model on real and random cifar10 images
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy.random import rand
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
import image_utils
import matplotlib.pyplot as plt
from PIL import Image

 
# define the standalone discriminator model
def define_discriminator32(in_shape=(32,32,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone discriminator model
def define_discriminator64(in_shape=(64,64,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(128, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(512, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator32(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the standalone generator model
def define_generator64(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# load and prepare cifar10 training images
def load_real_samples_cifar10():
    # load cifar10 dataset
    (trainX, _), (_, _) = load_data()
    
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

def trim_classes(images, labels, max_classes=10):
    return images[0:900*max_classes], labels[0:900*max_classes]



# load and prepare cifar10 training images
def load_real_samples_imagenet64(n_classes=10):
    # load imagenet64 dataset
    images, labels = image_utils.load_images(path_root="")
    images, labels = trim_classes(images, labels, n_classes)
    (trainX, _), (_, _) = image_utils.split_data(images, labels)
    
    
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
#     X = (X - 127.5) / 127.5
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# train the discriminator model
def train_discriminator(model, dataset, n_iter=20, n_batch=128):
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator on real samples
        _, real_acc = model.train_on_batch(X_real, y_real)
        # generate 'fake' examples
#         X_fake, y_fake = generate_fake_samples32(half_batch)
        X_fake, y_fake = generate_fake_samples(half_batch)
        # update discriminator on fake samples
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        # summarize performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))
        
# define the discriminator model
# model = define_discriminator32()
model = define_discriminator64()
# load image data
# dataset = load_real_samples_cifar10()
dataset = load_real_samples_imagenet64()

# fit the model
train_discriminator(model, dataset)

>1 real=12% fake=0%
>2 real=100% fake=0%
>3 real=100% fake=0%
>4 real=100% fake=0%
>5 real=100% fake=0%
>6 real=100% fake=20%
>7 real=100% fake=84%
>8 real=100% fake=100%
>9 real=100% fake=100%
>10 real=100% fake=100%
>11 real=100% fake=100%
>12 real=100% fake=100%
>13 real=100% fake=100%
>14 real=100% fake=100%
>15 real=100% fake=100%
>16 real=100% fake=100%
>17 real=100% fake=100%
>18 real=100% fake=100%
>19 real=100% fake=100%
>20 real=97% fake=100%


In [ ]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import ReLU
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot
import image_utils
from PIL import Image
import numpy as np
from numpy.random import random


# define the standalone discriminator model
def define_discriminator32(in_shape=(32,32,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone discriminator model
def define_discriminator64v1(in_shape=(64,64,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(128, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(512, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.00005, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone discriminator model
def define_discriminator64(in_shape=(64,64,3)):
    model = Sequential()
    init = RandomNormal(mean=0.0, stddev=0.02)
    
    # normal
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=init, input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(512, (3,3), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.00005, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator32(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the standalone generator model
def define_generator64(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the standalone generator model
def define_generator64v2(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 1024 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 1024)))
    # upsample to 8x8
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the standalone generator model
def define_generator64v3(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 1024 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(ReLU())
    model.add(Reshape((4, 4, 1024)))
    # upsample to 8x8
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
    model.add(ReLU())
    # upsample to 16x16
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(ReLU())
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(ReLU())
    # upsample to 64x64
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'))
    model.add(ReLU())
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the standalone generator model
def define_generator64v4(latent_dim):
    model = Sequential()
    init = RandomNormal(mean=0.0, stddev=0.02)
    
    # foundation for 4x4 image
    n_nodes = 1024 * 4 * 4
    model.add(Dense(n_nodes,  kernel_initializer=init, input_dim=latent_dim))
    model.add(ReLU())
    model.add(Reshape((4, 4, 1024)))
    # upsample to 8x8
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(ReLU())
    # upsample to 16x16
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(ReLU())
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(ReLU())
    # upsample to 64x64
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), kernel_initializer=init, padding='same'))
    model.add(ReLU())
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model
 
    
# load and prepare cifar10 training images
def load_real_samples32():
    # load cifar10 dataset
    (trainX, _), (_, _) = load_data()
    
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

def trim_classes(images, labels, max_classes=10):
    return images[0:900*max_classes], labels[0:900*max_classes]

def resize_images(images, size=(28,28)):
    temp = []
    for img in images:
        temp.append(np.array(Image.fromarray(img).resize(size, Image.ANTIALIAS)))
    return np.asarray(temp)

# load and prepare cifar10 training images
def load_real_samples64(n_classes=1):
    # load imagenet64 dataset
    images, labels = image_utils.load_images(path_root="")
    images, labels = trim_classes(images, labels, 1)
    (trainX, _), (_, _) = image_utils.split_data(images, labels, train=1)
#     trainX = resize_images(trainX, size=(32,32))
    
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

 
# example of smoothing class=1 to [0.7, 1.2]
def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)
    
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
#     y = smooth_positive_labels(y)
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10000, n_batch=100):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator64()
# create the generator
g_model = define_generator64(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples64()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)


# images, labels = image_utils.load_images(path_root="")
# images, labels = trim_classes(images, labels, 1)
# (trainX, _), (_, _) = image_utils.split_data(images, labels, train=1)
# trainX = resize_images(trainX, size=(32,32))

# print(trainX.shape)
# pyplot.imshow(trainX[899])

>1, 1/9, d1=0.679, d2=0.695 g=0.692
>1, 2/9, d1=0.641, d2=0.696 g=0.691
>1, 3/9, d1=0.600, d2=0.697 g=0.690
>1, 4/9, d1=0.557, d2=0.700 g=0.687
>1, 5/9, d1=0.503, d2=0.707 g=0.680
>1, 6/9, d1=0.442, d2=0.726 g=0.662
>1, 7/9, d1=0.392, d2=0.769 g=0.628
>1, 8/9, d1=0.326, d2=0.863 g=0.569
>1, 9/9, d1=0.281, d2=0.998 g=0.516
>2, 1/9, d1=0.295, d2=1.094 g=0.510
>2, 2/9, d1=0.262, d2=0.970 g=0.609
>2, 3/9, d1=0.313, d2=0.754 g=0.789
>2, 4/9, d1=0.215, d2=0.567 g=0.988
>2, 5/9, d1=0.232, d2=0.498 g=1.079
>2, 6/9, d1=0.220, d2=0.535 g=0.990
>2, 7/9, d1=0.189, d2=0.631 g=0.862
>2, 8/9, d1=0.132, d2=0.707 g=0.782
>2, 9/9, d1=0.142, d2=0.779 g=0.712
>3, 1/9, d1=0.208, d2=0.892 g=0.647
>3, 2/9, d1=0.221, d2=1.050 g=0.583
>3, 3/9, d1=0.266, d2=1.311 g=0.528
>3, 4/9, d1=0.250, d2=1.409 g=0.568
>3, 5/9, d1=0.331, d2=1.178 g=0.769
>3, 6/9, d1=0.491, d2=0.771 g=1.180
>3, 7/9, d1=0.525, d2=0.391 g=1.645
>3, 8/9, d1=0.694, d2=0.352 g=1.438
>3, 9/9, d1=0.664, d2=0.483 g=1.053
>4, 1/9, d1=0.581, d2=0.579 

### ThoughtVIZ's  Baseline AC-GAN

**Download model**

In [ ]:
!mkdir -p ThoughtViz/models
!wget -q https://raw.githubusercontent.com/ptirupat/ThoughtViz/master/training/models/ac_gan.py -P ThoughtViz/models

**Import libraries**

In [ ]:
import os
import numpy as np
import image_utils
from PIL import Image
import math


from random import randint
from PIL import Image
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from ThoughtViz.models import ac_gan
import matplotlib.pyplot as plt
import tensorflow as tf
import image_utils

In [ ]:
import os.path
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf
import math
import sys
from keras.utils import to_categorical


MODEL_DIR = '/tmp/imagenet'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
softmax = None


# Call this function with list of images. Each of elements should be a
# numpy array with values ranging from 0 to 255.
def get_inception_score(images, splits=10):
  assert(type(images) == list)
  assert(type(images[0]) == np.ndarray)
  assert(len(images[0].shape) == 3)
  assert(np.max(images[0]) > 10)
  assert(np.min(images[0]) >= 0.0)
  inps = []
  for img in images:
    img = img.astype(np.float32)
    inps.append(np.expand_dims(img, 0))
  bs = 1
  with tf.compat.v1.Session() as sess:
    preds = []
    n_batches = int(math.ceil(float(len(inps)) / float(bs)))
    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * bs):min((i + 1) * bs, len(inps))]
        inp = np.concatenate(inp, 0)
        pred = sess.run(softmax, {'ExpandDims:0': inp})
        preds.append(pred)
    preds = np.concatenate(preds, 0)
    scores = []
    for i in range(splits):
      part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
      kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
      kl = np.mean(np.sum(kl, 1))
      scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)


# This function is called automatically.
def _init_inception():
  global softmax
  if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(MODEL_DIR, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(MODEL_DIR)
  with tf.compat.v1.gfile.FastGFile(os.path.join(
      MODEL_DIR, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
  # Works with an arbitrary minibatch size.
  with tf.compat.v1.Session() as sess:
    pool3 = sess.graph.get_tensor_by_name('pool_3:0')
    ops = pool3.graph.get_operations()
    for op_idx, op in enumerate(ops):
        for o in op.outputs:
            shape = o.get_shape()
            shape = [s for s in shape]
            new_shape = []
            for j, s in enumerate(shape):
                if s == 1 and j == 0:
                    new_shape.append(None)
                else:
                    new_shape.append(s)
            o.set_shape(tf.TensorShape(new_shape))
    w = sess.graph.get_operation_by_name("softmax/logits/MatMul").inputs[1]
    logits = tf.matmul(tf.squeeze(pool3, [1, 2]), w)
    softmax = tf.nn.softmax(logits)


if softmax is None:
  _init_inception()

**Helper Functions**

In [ ]:
def trim_classes(images, labels, max_classes=10):
  return images[0:900*max_classes], labels[0:900*max_classes]

def combine_rgb_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], 3),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1], :] = img[:, :, :]
    return image

def load_data(num_classes):
    images, labels = image_utils.load_images(path_root="")
    images, labels = trim_classes(images, labels, num_classes)
    (x_train, y_train), (x_test, y_test) = image_utils.split_data(images, labels)
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_test = (x_test.astype(np.float32) - 127.5) / 127.5
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return (x_train, y_train), (x_test, y_test)

def train_gan(input_noise_dim, batch_size, epochs, model_save_dir, output_dir, num_classes):
    
    (x_train, y_train), (x_test, y_test) = load_data(num_classes)
    print(y_train.shape)
    adam_lr = 0.001
    adam_beta_1 = 0.5

    d = ac_gan.discriminator_model_rgb((64,64), num_classes)
    d_optim = Adam(lr=adam_lr, beta_1=adam_beta_1)
    d.compile(loss=['binary_crossentropy','categorical_crossentropy'], optimizer=d_optim)
    d.trainable = True

    g = ac_gan.generator_model_rgb(input_noise_dim + num_classes)
    g_optim = Adam(lr=adam_lr, beta_1=adam_beta_1)
    g.compile(loss='categorical_crossentropy', optimizer=g_optim)

    d_on_g = ac_gan.generator_containing_discriminator(input_noise_dim + num_classes, g, d)
    d_on_g.compile(loss=['binary_crossentropy','categorical_crossentropy'], optimizer=g_optim)

    g.summary()
    d.summary()
    
    for epoch in range(epochs):
        print("Epoch is ", epoch)

        print("Number of batches", int(x_train.shape[0]/batch_size))

        for index in range(int(x_train.shape[0]/batch_size)):
            # generate noise from a normal distribution
            noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

            random_labels = [randint(0, 9) for i in range(batch_size)]

            one_hot_vectors = [to_categorical(label, 10) for label in random_labels]

            conditioned_noise = []
            for i in range(batch_size):
                conditioned_noise.append(np.append(noise[i], one_hot_vectors[i]))
            conditioned_noise = np.array(conditioned_noise)

            # get real images and corresponding labels
            real_images = x_train[index * batch_size:(index + 1) * batch_size]
            real_labels = y_train[index * batch_size:(index + 1) * batch_size]

            # generate fake images using the generator
            generated_images = g.predict(conditioned_noise, verbose=0)

            # discriminator loss of real images
            d_loss_real = d.train_on_batch(real_images, [np.array([1] * batch_size), np.array(real_labels)])
            # discriminator loss of fake images
            d_loss_fake = d.train_on_batch(generated_images, [np.array([0] * batch_size), np.array(one_hot_vectors).reshape(batch_size, num_classes)])
            d_loss = (d_loss_fake[0] + d_loss_real[0]) * 0.5

            # save generated images at intermediate stages of training
            if index % 250 == 0:
                image = combine_rgb_images(generated_images)
                image = image * 255.0
                img_save_path = os.path.join(output_dir, str(epoch) + "_g_" + str(index) + ".png")
                Image.fromarray(image.astype(np.uint8)).save(img_save_path)

            d.trainable = False
            # generator loss
            g_loss = d_on_g.train_on_batch(conditioned_noise, [np.array([1] * batch_size), np.array(one_hot_vectors).reshape(batch_size, num_classes)])
            d.trainable = True

        # test_image_count = 50000
        test_image_count = x_train.shape[0]
        test_noise = np.random.uniform(-1, 1, (test_image_count, input_noise_dim))
        test_labels = [randint(0, 9) for i in range(test_image_count)]
        one_hot_vectors_test = [to_categorical(label, 10) for label in test_labels]

        conditioned_noise_test = []
        for i in range(test_image_count):
            conditioned_noise_test.append(np.append(test_noise[i], one_hot_vectors_test[i]))
        conditioned_noise_test = np.array(conditioned_noise_test)

        test_images = g.predict(conditioned_noise_test, verbose=0)
        test_images = test_images * 255.0
        
        if epoch % 50 == 0:        
            inception_score = get_inception_score([test_image for test_image in test_images], splits=10)

        print("Epoch %d d_loss : %f" % (epoch, d_loss))
        print("Epoch %d g_loss : %f" % (epoch, g_loss[0]))
        print("Epoch %d inception_score : %f" % (epoch, inception_score[0]))

        # save generator and discriminator models along with the weights
        g.save(os.path.join(model_save_dir, 'generator_' + str(epoch)), overwrite=True, include_optimizer=True)
        d.save(os.path.join(model_save_dir, 'discriminator_' + str(epoch)), overwrite=True, include_optimizer=True)

**Train GAN**

In [ ]:
batch_size =50
run_id = 1
n_epochs = 500
input_dim = 100
n_classes=10

model_save_dir = os.path.join('./saved_models/baseline_acgan/', 'run_' + str(run_id))
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

output_dir = os.path.join('./outputs/baseline_acgan/', 'run_' + str(run_id))
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

train_gan(input_dim, batch_size, n_epochs, model_save_dir, output_dir, n_classes)

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

def resize_images(images, size=(28,28)):
  temp = []
  for img in images:
    temp.append(np.array(Image.fromarray(img).resize(size, Image.ANTIALIAS)))
  return np.asarray(temp)

def trim_classes(images, labels, max_classes=10):
  return images[0:900*max_classes], labels[0:900*max_classes]

In [ ]:
images, labels = image_utils.load_images()
images = rgb2gray(images)
print(images.shape)
images = resize_images(images)
print(images.shape)
(x_train, y_train), (x_test, y_test) = image_utils.split_data(images, labels)

**Download Trained Model**

In [ ]:
!mkdir -p /content/models/ThoughtViz
!cp /content/gdrive/MyDrive/EEG2Image/models/ThoughtViz/baseline\ acgan/run_1/generator_100/saved_model.pb  /content/models/ThoughtViz/
!mv /content/models/ThoughtViz/saved_model.pb /content/models/ThoughtViz/generator.pb
!cp /content/gdrive/MyDrive/EEG2Image/models/ThoughtViz/baseline\ acgan/run_1/discriminator_100/saved_model.pb  /content/models/ThoughtViz/
!mv /content/models/ThoughtViz/saved_model.pb /content/models/ThoughtViz/discriminator.pb

In [ ]:
!mkdir -p /content/models/ThoughtViz
!cp -r /content/gdrive/MyDrive/EEG2Image/models/ThoughtViz/baseline\ acgan/run_1/generator_100 /content/models/ThoughtViz/
!cp -r /content/gdrive/MyDrive/EEG2Image/models/ThoughtViz/baseline\ acgan/run_1/discriminator_100 /content/models/ThoughtViz/


**Load Model**

In [ ]:
import tensorflow as tf
import pickle
import random

import keras.backend as K
from PIL import Image
from keras.models import load_model
from keras.utils import to_categorical


In [ ]:
K.set_learning_phase(False)

input_noise_dim = 100
batch_size = 50

noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

random_labels = np.random.randint(0, 10, batch_size)

conditioned_noise = []
for i in range(batch_size):
    conditioned_noise.append(np.append(noise[i], to_categorical(0, 10)))
conditioned_noise = np.array(conditioned_noise)

g = load_model("/content/models/ThoughtViz/generator_100")

# generate images using the generator
generated_images = g.predict(conditioned_noise, verbose=0)

image = combine_rgb_images(generated_images)
image = image * 127.5 + 127.5
plt.imshow(image)
img = Image.fromarray(image.astype(np.uint8))
img.show()
img.save("test.png")

In [ ]:
import pickle
import random

import keras.backend as K
from PIL import Image
from keras.models import load_model
from keras.utils import to_categorical

from layers.mog_layer import *
from utils.image_utils import *


class Tests():

    def test_deligan_baseline(self, generator_model):
        K.set_learning_phase(False)

        input_noise_dim = 100
        batch_size = 50

        noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

        random_labels = np.random.randint(0, 10, batch_size)

        conditioned_noise = []
        for i in range(batch_size):
            conditioned_noise.append(np.append(noise[i], to_categorical(random_labels[i], 10)))
        conditioned_noise = np.array(conditioned_noise)

        g = load_model(generator_model, custom_objects={'MoGLayer': MoGLayer})

        # generate images using the generator
        generated_images = g.predict(conditioned_noise, verbose=0)

        image = combine_rgb_images(generated_images)
        image = image * 127.5 + 127.5
        img = Image.fromarray(image.astype(np.uint8))
        img.show()

    def test_deligan_final(self, generator_model, classifier_model, eeg_pkl_file):
        K.set_learning_phase(False)

        # load EEG data
        eeg_data = pickle.load(open(eeg_pkl_file, "rb"), encoding='bytes')
        classifier = load_model(classifier_model)

        x_test = eeg_data[b'x_test']
        y_test = eeg_data[b'y_test']
        y_test = [np.argmax(y) for y in y_test]
        layer_index = 9

        # keras way of getting the output from an intermediate layer
        get_nth_layer_output = K.function([classifier.layers[0].input], [classifier.layers[layer_index].output])

        layer_output = get_nth_layer_output([x_test])[0]

        input_noise_dim = 100
        batch_size = 50

        noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

        random_labels = np.random.randint(0, 10, batch_size)

        eeg_feature_vectors = [layer_output[random.choice(np.where(y_test == random_label)[0])] for random_label in random_labels]

        noises, conditionings = [], []
        for i in range(batch_size):
            noises.append(noise[i])
            conditionings.append(eeg_feature_vectors[i])

        g = load_model(generator_model, custom_objects={'MoGLayer': MoGLayer})

        # generate images using the generator
        generated_images = g.predict([np.array(noises), np.array(conditionings)], verbose=0)

        image = combine_rgb_images(generated_images)
        image = image * 127.5 + 127.5
        img = Image.fromarray(image.astype(np.uint8))
        img.show()


if __name__ == '__main__':
    tests = Tests()
    #tests.test_deligan_baseline('../models/gan_models/baseline/deligan/image/generator.model')
    tests.test_deligan_final('../models/gan_models/final/image/generator.model',
                       '../models/eeg_models/image/run_final.h5',
                       '../data/eeg/image/data.pkl')

# ThoughtViz

**Complete the following steps before running the cells below**

1.
Change runtime to GPU 

>Runtime > Change runtime type > GPU

2.
For each of the following links, add a Google Drive shortcut to your Drive root folder(My Drive):

>https://drive.google.com/file/d/1atP9CsjWIT-hg3fX--fcC1hg0uvg9bEH/view

>https://drive.google.com/file/d/1x32IulYeBVmkshEKweijMX3DK1Wu8odx/view

>https://drive.google.com/file/d/1cq8RTBiwqO-Jo0TZjBNlRHZEhjKDknKP/view

>https://drive.google.com/file/d/1U9qtN1SlOS3dzd2BwWWHhJiMz_0yNW9U/view

>https://drive.google.com/file/d/1uFFhvTsU2nmdaecR2WPWsiGJfgI3as1_/view

**Mount drive and download project**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!git clone "https://github.com/ptirupat/ThoughtViz.git" "/content/ThoughtViz"

**Unzip project files**

In [ ]:
!unzip -q "/content/gdrive/My Drive/data.zip" -d "/content/ThoughtViz" 
!unzip -q "/content/gdrive/My Drive/images.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/eeg_models.zip" -d "/content/ThoughtViz/models" 
!unzip -q "/content/gdrive/My Drive/trained_classifier_models.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/gan_models.zip" -d "/content/ThoughtViz/models"
!echo "Download Finished!"

**Update outdated module calls and make code compatible with colab environment**

In [ ]:
# Update Keras module structure
!sed -i "s/keras.layers/keras/" ThoughtViz/layers/mog_layer.py

# Change System Path
!sed -i "1i import sys\nsys.path.insert(0,'/content/ThoughtViz')\n" /content/ThoughtViz/testing/test.py
!sed -i "s/\..\//\/content\/ThoughtViz\//" ThoughtViz/testing/test.py

# Rename GAN model directory to match code
!mv /content/ThoughtViz/models/gan_models/thoughtviz/ /content/ThoughtViz/models/gan_models/final/

**Test code**

Note: Images are not saved by default. 
>To save the output image, you must edit the *ThoughtViz/testing/test.py* file and add the line **img.save('output.png')** after *img.show()* in the *test_deligan_final* function. 

In [ ]:
!python3 ThoughtViz/testing/test.py

# EEG decoding to images - Converting Data

**Convert to NPY**

Code provided by: *https://github.com/perceivelab/eeg_visual_classification/blob/main/eeg_signal_classification.py*

In [ ]:
import torch

def pth_to_npy(path):
      torch_file = torch.load(path)

      torch_dataset = torch_file["dataset"]
      torch_labels = torch_file["labels"]
      torch_images = torch_file["images"]
      torch_size = len(torch_dataset)

      return torch_dataset, torch_labels, torch_images, torch_size

torch_dataset, torch_labels, torch_images, torch_size = pth_to_npy(data_path)
# # Dataset class
# class EEGDataset:
    
#     # Constructor
#     def __init__(self, eeg_signals_path):
#         # Load EEG signals
#         loaded = torch.load(eeg_signals_path)
#         if opt.subject!=0:
#             self.data = [loaded['dataset'][i] for i in range(len(loaded['dataset']) ) if loaded['dataset'][i]['subject']==opt.subject]
#         else:
#             self.data=loaded['dataset']        
#         self.labels = loaded["labels"]
#         self.images = loaded["images"]
        
#         # Compute size
#         self.size = len(self.data)

#     # Get size
#     def __len__(self):
#         return self.size

#     # Get item
#     def __getitem__(self, i):
#         # Process EEG
#         eeg = self.data[i]["eeg"].float().t()
#         eeg = eeg[opt.time_low:opt.time_high,:]

#         if opt.model_type == "model10":
#             eeg = eeg.t()
#             eeg = eeg.view(1,128,opt.time_high-opt.time_low)
#         # Get label
#         label = self.data[i]["label"]
#         # Return
#         return eeg, label

In [ ]:
print("size:", torch_size)

In [ ]:
# print("dataset 0:", torch_dataset[0])
eeg = torch_dataset[0]["eeg"]
image_num = torch_dataset[0]["image"]
label = torch_dataset[0]["label"]
subject = torch_dataset[0]["subject"]

print("eeg data:", eeg)
print("image number:", image_num)
print("image label:", label)
print("participant number:", subject)

trimmed_eeg = eeg.float().t()[40:460]
print("eeg data trimmed:", trimmed_eeg)

numpy_eeg = trimmed_eeg.numpy()
print("eeg data trimmed+numpy:", numpy_eeg)


In [ ]:
print("label 0:", torch_labels[0])
print("label 1:", torch_labels[1])

In [ ]:
print("image 0:", torch_images[0])
print("image 1:", torch_images[1])

for i in range(0,10):
  print(torch_images[i])

# Visualization

**Download and Import Packages**

In [ ]:
!pip install -U mne
!pip install pyEDFlib

In [ ]:
%matplotlib inline
import mne
import os
import numpy as np
from pyedflib import highlevel

**Define Visualization Functions**

In [ ]:
def create_edf(npy_path, name):
  data = np.load(npy_path)
  # [128,440,11965]
  signals = list(data.T[:,:,:])
  channel_names = list('ch'+str(i) for i in range(0,128))
  signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=1000)
  header = highlevel.make_header()
  highlevel.write_edf(name, signals, signal_headers, header)

def create_edf2(npy_path, name):
  data = np.load(npy_path)
  # [128,11965,440]
  signals = list(data.transpose(2,0,1)[0:ch,0:100,:])
  channel_names = list('ch'+str(i) for i in range(0,128))
  signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=1000)
  header = highlevel.make_header()
  highlevel.write_edf(name, signals, signal_headers, header)

def visualizeData(path):
  raw = mne.io.read_raw_edf(path,preload=True);
  print(raw)
  raw.plot_psd(area_mode='range', fmin=15, fmax=70.0, show=False, average=True);
  raw.plot(duration=2, scalings=0.000002)

In [ ]:
data_path = '/content/gdrive/MyDrive/EEG2Image/data/eeg_14_70.npy'
test_path = '/content/test_file.edf'
rand_path = '/content/edf_file.edf'

In [ ]:
print(data.shape)
print(data[::1,1].shape)
print(data[::1,2].shape)
print(data[1::].shape)
print(data[2::].shape)
print(data.T[0,0].shape)
print(data.T.shape)
print(data.T[:,:,0:10].shape)
print(data.T[:,:,0:10].reshape(128,440*10).shape)
print(data.transpose(2,0,1).shape)
print(data.transpose(2,0,1)[:,0:10,:].shape)
data.transpose(2,0,1)[0:10,0:10,:].reshape((128,440*10)).shape


In [ ]:
# write an edf file
ch = 5
signals = list(data.T[0:ch,:,0:100].reshape(ch,440*100))
channel_names = list('ch'+str(i) for i in range(0,ch))
signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=1000)
header = highlevel.make_header(patientname='patient_x', gender='Female')
highlevel.write_edf('test_file.edf', signals, signal_headers, header)

visualizeData(test_path)

In [ ]:
# write an edf file
ch = 5
signals = list(data.transpose(2,0,1)[0:ch,0:100,:])
channel_names = list('ch'+str(i) for i in range(0,ch))
signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=1000)
header = highlevel.make_header()
highlevel.write_edf('test_file.edf', signals, signal_headers, header)

visualizeData(test_path)

In [ ]:
# write an edf file
ch = 50
signals = list(data.transpose(2,0,1)[0:ch,0:100,:])
channel_names = list('ch'+str(i) for i in range(0,ch))
signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=1000)
header = highlevel.make_header()
highlevel.write_edf('test_file.edf', signals, signal_headers, header)

visualizeData(test_path)

In [ ]:
visualizeData(test_path)

In [ ]:
visualizeData(test_path)

**LSTM stuff**

In [ ]:
# Code goes here #

**GAN stuff**

In [ ]:
# Code goes here #

**Test Code**

In [ ]:
# Code goes here #